## Lazynwb

In [1_polars.ipynb](1_polars.ipynb), we saw predicate pushdown in action to reduce the amount of
data polars reads from a parquet file:

scan parquet file -> `LazyFrame` -> write query -> `.collect()` -> polars optimizes query &
fetches data -> `DataFrame`

The `lazynwb` package allows us to work with NWB files in a similar way.

It works with single files, or multiple files, virtually concatenating tables across files:

In [9]:
import pathlib

import lazynwb
import polars as pl


nwb_paths = list(pathlib.Path('data/dynamicrouting_datacube_v0.0.268/nwb').glob('*.nwb'))
print(f"Found {len(nwb_paths)} NWB files")

Found 252 NWB files


To see the available paths within one NWB file:

In [10]:
lazynwb.get_internal_paths(nwb_paths[0])

{'/acquisition/frametimes_eye_camera/timestamps': <HDF5 dataset "timestamps": shape (267399,), type "<f8">,
 '/acquisition/frametimes_front_camera/timestamps': <HDF5 dataset "timestamps": shape (267204,), type "<f8">,
 '/acquisition/frametimes_side_camera/timestamps': <HDF5 dataset "timestamps": shape (267374,), type "<f8">,
 '/acquisition/lick_sensor_events/data': <HDF5 dataset "data": shape (2734,), type "<f8">,
 '/acquisition/lick_sensor_events/timestamps': <HDF5 dataset "timestamps": shape (2734,), type "<f8">,
 '/intervals/aud_rf_mapping_trials': <HDF5 group "/intervals/aud_rf_mapping_trials" (10 members)>,
 '/intervals/epochs': <HDF5 group "/intervals/epochs" (9 members)>,
 '/intervals/performance': <HDF5 group "/intervals/performance" (21 members)>,
 '/intervals/trials': <HDF5 group "/intervals/trials" (48 members)>,
 '/intervals/vis_rf_mapping_trials': <HDF5 group "/intervals/vis_rf_mapping_trials" (12 members)>,
 '/processing/behavior/dlc_eye_camera': <HDF5 group "/processing/

Access one of the paths as a `LazyFrame` across all NWB files:

- note: `infer_schema_length` optionally limits how many files are scanned to get the schema of the table, since in this case the schema is the same across all files

In [11]:
lf = lazynwb.scan_nwb(nwb_paths, table_path='/intervals/performance', infer_schema_length=1)
lf

We can examine the schema of the table at this point:

In [12]:
lf.collect_schema()

Schema([('aud_dprime', Float64),
        ('aud_nontarget_response_rate', Float64),
        ('aud_target_response_rate', Float64),
        ('block_index', Int64),
        ('catch_response_rate', Float64),
        ('cross_modality_dprime', Float64),
        ('false_alarm_rate', Float64),
        ('hit_rate', Float64),
        ('id', Int64),
        ('is_first_block_aud', Boolean),
        ('n_contingent_rewards', Int64),
        ('n_hits', Int64),
        ('n_responses', Int64),
        ('n_trials', Int64),
        ('rewarded_modality', String),
        ('signed_cross_modality_dprime', Float64),
        ('start_time', Float64),
        ('stop_time', Float64),
        ('vis_dprime', Float64),
        ('vis_nontarget_response_rate', Float64),
        ('vis_target_response_rate', Float64),
        ('_nwb_path', String),
        ('_table_path', String),
        ('_table_index', UInt32)])

note: `_nwb_path` and `_table_row_index` are not columns in the NWB table on disk -- they're added by `lazynwb` to identify the source of each row in a table that spans multiple NWBs

Now we can carry out the same query as we did in [1_polars.ipynb](1_polars.ipynb) for parquet files:

In [13]:
lazynwb.scan_nwb(nwb_paths, table_path='/general/subject', infer_schema_length=1).collect_schema()

Schema([('age', String),
        ('date_of_birth', String),
        ('genotype', String),
        ('sex', String),
        ('species', String),
        ('strain', String),
        ('subject_id', String),
        ('_nwb_path', String),
        ('_table_path', String),
        ('_table_index', UInt32)])

In [14]:
lf = (
    lazynwb.scan_nwb(nwb_paths, table_path='/intervals/performance', infer_schema_length=1)

    # metadata containing session kewords and subject ID is spread out in the NWB file so we have to combine the following two tables:
    .join(
        lazynwb.scan_nwb(nwb_paths, table_path='/general', infer_schema_length=1).select('_nwb_path', 'keywords', 'session_id'),
        on='_nwb_path',
        how='left',
    )
    .join(
        lazynwb.scan_nwb(nwb_paths, table_path='/general/subject', infer_schema_length=1).select('_nwb_path', 'subject_id'),
        on='_nwb_path',
        how='left',
    )
    
    .filter(
        pl.col('keywords').list.contains('production'),
        pl.col('signed_cross_modality_dprime').drop_nans().mean().over('session_id') > 2.0, # <-- for each row, only consider the other rows with the same session_id
    )
    .with_columns(
        (pl.col('stop_time') - pl.col('start_time')).alias('block_duration'),
    )
    .select('block_index', 'block_duration', 'signed_cross_modality_dprime', 'subject_id', '_nwb_path')
    .sort('_nwb_path', 'block_index')
)
lf

In [15]:
lf.collect()

Getting multi-NWB /general/subject table:  19%|███████▍                               | 48/252 [00:01<00:08, 24.10NWB/s]Table '/intervals/performance' not found in data/dynamicrouting_datacube_v0.0.268/nwb/721536_2024-05-16.nwb

Getting multi-NWB /general/subject table:  55%|████████████████████▊                 | 138/252 [00:08<00:13,  8.66NWB/s]

Getting multi-NWB /general/subject table:  64%|████████████████████████▍             | 162/252 [00:10<00:07, 12.18NWB/s]


Getting multi-NWB /general/subject table:  68%|█████████████████████████▊            | 171/252 [00:10<00:05, 15.03NWB/s]

Getting multi-NWB /general/subject table:  71%|██████████████████████████▉           | 179/252 [00:11<00:04, 15.32NWB/s]

Getting multi-NWB /general/subject table:  72%|███████████████████████████▎          | 181/252 [00:11<00:06, 10.78NWB/s]

Getting multi-NWB /general/subject table:  75%|████████████████████████████▋         | 190/252 [00:12<00:04, 13.08NWB/s]Table '/intervals/performance' not found

block_index,block_duration,signed_cross_modality_dprime,subject_id,_nwb_path
i64,f64,f64,str,str
0,603.68899,2.224615,"""703880""","""/root/capsule/data/dynamicrout…"
1,601.10347,1.769163,"""703880""","""/root/capsule/data/dynamicrout…"
2,604.8567,2.639312,"""703880""","""/root/capsule/data/dynamicrout…"
3,602.07098,2.102846,"""703880""","""/root/capsule/data/dynamicrout…"
4,605.50716,2.626949,"""703880""","""/root/capsule/data/dynamicrout…"
5,597.18357,1.232238,"""703880""","""/root/capsule/data/dynamicrout…"


Not as fast as accessing the parquet files (which contain the same data) but much faster than looping over NWB files and opening the tables with the pynwb package.

The largest performance gains will be seen for tables that contain array data in columns, for example the `units` table in ecephys NWBs:


In [16]:
lf = lazynwb.scan_nwb(nwb_paths, table_path='/units', infer_schema_length=1)
{k:v for k,v in lf.collect_schema().items() if k in ('waveform_mean', 'waveform_sd', 'spike_times', 'obs_intervals')}

{'obs_intervals': List(Array(Float64, shape=(2,))),
 'spike_times': List(Float64),
 'waveform_mean': Array(Float64, shape=(210, 384)),
 'waveform_sd': Array(Float64, shape=(210, 384))}

The `waveform_mean` and `waveform_sd` columns contain a `[210 x 384]` array in each row, and each row of the `spike_times` column contains a list with thousands of values.

Opening the units table with pynwb would read all of this data into memory, which is slow and wasteful.

With lazynwb we can filter rows based on metadata columns, then select the array data we want for the remaining rows:

In [17]:
(
    lf  
    .filter(
        pl.col('location').str.starts_with('CA1'),
        pl.col('activity_drift') <= 0.01,
        pl.col('amplitude_cutoff') <= 0.1,
        pl.col('presence_ratio') >= 0.7,
        pl.col('isi_violations_ratio') <= 0.5,
        pl.col('decoder_label') != 'noise',
    )
    .select('unit_id', 'location', 'spike_times', '_nwb_path')
    .collect()
)

Getting multi-NWB /units table:   0%|                                                          | 0/252 [00:00<?, ?NWB/s]

Getting multi-NWB /units table: 100%|██████████████████████████████████████████████████| 64/64 [00:22<00:00,  2.84NWB/s]


unit_id,location,spike_times,_nwb_path
str,str,list[f64],str
"""759434_2025-02-06_D-33""","""CA1""","[20.334285, 20.424817, … 7027.65843]","""/root/capsule/data/dynamicrout…"
"""759434_2025-02-06_D-73""","""CA1""","[20.551917, 20.967581, … 7028.016428]","""/root/capsule/data/dynamicrout…"
"""644864_2023-02-02_C-159""","""CA1""","[512.379566, 512.767995, … 5154.944709]","""/root/capsule/data/dynamicrout…"
"""726088_2024-06-18_E-96""","""CA1""","[19.944689, 19.975622, … 7104.380934]","""/root/capsule/data/dynamicrout…"
"""737403_2024-09-27_A-231""","""CA1""","[79.594146, 129.606797, … 7741.653876]","""/root/capsule/data/dynamicrout…"
…,…,…,…
"""628801_2022-09-19_C-271""","""CA1""","[34.477293, 34.83749, … 4872.188405]","""/root/capsule/data/dynamicrout…"
"""628801_2022-09-19_C-656""","""CA1""","[221.385049, 221.400315, … 4873.252029]","""/root/capsule/data/dynamicrout…"
"""664851_2023-11-16_B-320""","""CA1""","[20.63323, 23.156425, … 6691.391273]","""/root/capsule/data/dynamicrout…"


Behind the scenes, multiple operations take place:
1. the columns referenced in the `.filter()` operation are fetched for 252/252 NWB files
2. the filter operation is applied, which excludes some rows and some NWB files entirely
3. the columns requested in the `.select()` operation are fetched just for the filtered rows, which are found in 64/252 NWB files

---



## SQL -> polars -> NWB

For those who prefer to write SQL queries, `lazynwb` implements the following interface (https://docs.pola.rs/api/python/stable/reference/sql/python_api.html#sqlcontext):

In [ ]:
sql_context = lazynwb.get_sql_context(nwb_paths, infer_schema_length=1)

Scanning NWB files: 100%|███████████████████████| 1/1 [00:00<00:00, 89.01file/s]


<SQLContext [tables:20] at 0x7f276c254310>

In [25]:
sql_context.execute(
    """
    SELECT * FROM performance
    """,
    eager=True,
)

Getting multi-NWB intervals/performance table:   0%|                                           | 0/252 [00:00<?, ?NWB/s]

Getting multi-NWB intervals/performance table:  15%|█████                            | 39/252 [00:00<00:01, 126.31NWB/s]Table 'intervals/performance' not found in data/dynamicrouting_datacube_v0.0.268/nwb/754852_2025-02-13.nwb
Table 'intervals/performance' not found in data/dynamicrouting_datacube_v0.0.268/nwb/721536_2024-05-16.nwb
Table 'intervals/performance' not found in data/dynamicrouting_datacube_v0.0.268/nwb/724612_2024-07-25.nwb
Table 'intervals/performance' not found in data/dynamicrouting_datacube_v0.0.268/nwb/754852_2025-02-11.nwb
Table 'intervals/performance' not found in data/dynamicrouting_datacube_v0.0.268/nwb/660023_2023-08-08.nwb
Getting multi-NWB intervals/performance table:  60%|███████████████████             | 150/252 [00:01<00:00, 121.24NWB/s]Table 'intervals/performance' not found in data/dynamicrouting_datacube_v0.0.268/nwb/754852_2025-02-12.nwb
Table 'intervals/performance' not found in data/dynamicrouting_datacube_v0.0.268/nwb/724612_2024-07-22.nwb
Getting mul

block_index,cross_modality_dprime,stop_time,aud_nontarget_response_rate,signed_cross_modality_dprime,rewarded_modality,n_contingent_rewards,aud_dprime,catch_response_rate,n_trials,vis_target_response_rate,start_time,vis_dprime,false_alarm_rate,n_responses,id,hit_rate,vis_nontarget_response_rate,aud_target_response_rate,n_hits,is_first_block_aud,_nwb_path,_table_path,_table_index
i64,f64,f64,f64,f64,str,i64,f64,f64,i64,f64,f64,f64,f64,i64,i64,f64,f64,f64,i64,bool,str,str,u32
0,2.711324,1703.10128,0.235294,2.711324,"""vis""",19,-1.238442,5.285714,87,1.0,1103.37055,2.841431,0.232143,37,0,1.0,0.210526,0.25,24,false,"""/root/capsule/data/dynamicrout…","""intervals/performance""",0
1,-0.464653,2310.23841,0.933333,0.464653,"""aud""",13,-0.464653,4.416667,77,0.933333,1704.70261,-1.750263,0.777778,53,1,0.85,0.466667,0.866667,17,false,"""/root/capsule/data/dynamicrout…","""intervals/performance""",1
2,2.195563,2917.79243,0.190476,2.195563,"""vis""",21,-1.083821,5.714286,94,1.0,2310.4552,3.714755,0.229508,40,2,1.0,0.05,0.45,26,false,"""/root/capsule/data/dynamicrout…","""intervals/performance""",2
3,1.916772,3525.49664,1.0,-1.916772,"""aud""",17,0.166509,7.285714,74,0.533333,2918.12605,-1.210989,0.6,51,3,1.0,0.266667,1.0,22,false,"""/root/capsule/data/dynamicrout…","""intervals/performance""",3
4,0.833109,4135.9197,0.352941,0.833109,"""vis""",20,-1.537114,5.555556,87,1.0,3535.3048,3.02117,0.471698,50,4,1.0,0.166667,0.888889,25,false,"""/root/capsule/data/dynamicrout…","""intervals/performance""",4
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1,0.39923,3549.16307,0.105263,-0.39923,"""aud""",23,3.271206,4.777778,90,0.947368,2942.28823,0.022032,0.344828,43,1,1.0,0.0,1.0,23,false,"""/root/capsule/data/dynamicrout…","""intervals/performance""",1
2,0.408895,4155.73784,0.0,0.408895,"""vis""",25,0.0,4.888889,94,1.0,3549.24652,4.013713,0.316667,44,2,1.0,0.0,0.95,25,false,"""/root/capsule/data/dynamicrout…","""intervals/performance""",2
3,0.801629,4763.3301,0.263158,-0.801629,"""aud""",25,2.687389,8.166667,88,0.894737,4156.40506,-0.318075,0.403509,49,3,1.0,0.052632,1.0,25,false,"""/root/capsule/data/dynamicrout…","""intervals/performance""",3


Note: the metadata tables in `general` are currently missing from the SQL interface but will be added soon: 

In [26]:
sql_context.tables()

['aud_rf_mapping_trials',
 'dlc_eye_camera',
 'epochs',
 'eye_tracking',
 'facemap_front_camera',
 'facemap_side_camera',
 'frametimes_eye_camera',
 'frametimes_front_camera',
 'frametimes_side_camera',
 'lick_sensor_events',
 'licks',
 'lp_front_camera',
 'lp_side_camera',
 'performance',
 'quiescent_interval_violations',
 'rewards',
 'running_speed',
 'trials',
 'units',
 'vis_rf_mapping_trials']